## Getting started

In [ ]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

import os.path 

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import HTML

## *E. coli*

We will read an *E. coli* metabolic model. The model has been downloaded from http://bigg.ucsd.edu/models/e_coli_core.

In [ ]:
model = cobra.io.read_sbml_model(os.path.join('models','e_coli_core.xml'))

In [ ]:
model

In [ ]:
model.reactions[0]

Let's define the biommas function as the metabolic objective:

In [ ]:
model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')

In [ ]:
r_biomass = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
r_biomass

In [ ]:
model.objective = r_biomass

In [ ]:
model.objective.expression

In [ ]:
sol = model.optimize()
sol

In [ ]:
model.summary()

We can see that the cell growth equals 0.874 $h^{-1}$.

We can compare this with the result obtained using [Escher-FBA](https://sbrg.github.io/escher-fba/#/app):

In [ ]:
sol = model.optimize()
sol['EX_glc__D_e']

In [ ]:
sol = model.optimize()
sol['EX_o2_e']

### Growth medium

We can try changing the medium composition. As we see above, the cell consumes a significant amount of glucose (the flux through *EX_glc__D_e* equals 10 $mmol gDW^{-1} hr^{-1}$ -- millimole per gram dry weight per hour - default units in COBRA). What is the maximal possible uptake of glucose? Does the cell consume everything that is available?

In [ ]:
r_ex_glc = model.reactions.get_by_id('EX_glc__D_e')
r_ex_glc

We can see that the flux is negative. However, the reaction is reversed (glucose secretion)! In COBRA all *exchange* (*uptake* and *secretion*) reactions have such form. 

As we can see the cell consumes all the glucose that is available. Let's try to reduce the glucose in the medium.

In [ ]:
r_ex_glc.bounds

In [ ]:
r_ex_glc.bounds = (0, 1000.0)
model.optimize()

Oh no. This was too much. Let's be a little bit more permissive.

In [ ]:
r_ex_glc.bounds = (-1.0, 1000.0)

In [ ]:
model.optimize()

We can repeat a similar analysis in anaerobic conditions. We will remove the oxygen by setting its *uptake* bounds to 0:

In [ ]:
r_ex_o2 = model.reactions.get_by_id('EX_o2_e')
r_ex_o2

In [ ]:
r_ex_glc.bounds = (-10.0, 1000.0)
r_ex_o2.bounds = (0, 1000)

In [ ]:
model.summary()

Biomass production has dropped. Also, alternative pathway is now used for energy productions. The cell is now generating and secreting acetate (`ac`), formate (`for`) and ethanol (`etoh`). Before, the cell has been secreting water and carbon dioxide (cellular respiration). 

We can further analyse biomass production rate in dependence on oxygen and glucose:

In [ ]:
o2_max = 1500
glc_max = 1000

# we generate a sorted list of oxygen and glucose values
o2_range = np.arange(0, o2_max+1,100)
glc_range = np.arange(0, glc_max+1,100)

# a matrix with two rows; one for oxygen and one for glucose
B = np.zeros((len(o2_range),len(glc_range)))

# for each pair of values of O2 and glc we calculate and save the biomass production rate
for i,o2 in enumerate(o2_range):
    for j,glc in enumerate(glc_range):
        r_ex_o2.lower_bound = -o2 # setting the oxygen uptake value
        r_ex_glc.lower_bound = -glc # setting the glucose uptake value
        s = model.optimize()
        B[i,j] = s.objective_value # saving the result

In [ ]:
ax = sns.heatmap(B, xticklabels=glc_range, yticklabels=o2_range)
ax.invert_yaxis()
plt.xlabel("glc")
plt.ylabel("o2")
plt.show()

## FVA
FVA (Flux Variability Analysis) allows the analysis of flux ranges bringing the system to almost optimal biomass production. 

In [ ]:
from cobra.flux_analysis import flux_variability_analysis

In [ ]:
fva = flux_variability_analysis(model, fraction_of_optimum=0.9, loopless=True)

In [ ]:
fva

In [ ]:
model.optimize()
model.summary(fva=0.90)

### Genes in the model

In [ ]:
for r in model.reactions:
    print(r.gpr)

In [ ]:
model.genes.get_by_id('b4015')

## Recon3D

Recon3D presents a human metabolic model. The model has been downloaded from [https://github.com/SBRG/Recon3D](https://github.com/SBRG/Recon3D). The model is described in the paper available [here](http://dx.doi.org/10.1038/nbt.4072).

In [ ]:
model_rec3d = cobra.io.read_sbml_model(os.path.join('models','Recon3D.xml'))

In [ ]:
model_rec3d

Some of the model attributes:

In [ ]:
model_rec3d.genes[0]

In [ ]:
model_rec3d.metabolites[:3]

In [ ]:
model_rec3d.reactions[:3]

The model summary can be obtained with the method [summary](https://cobrapy.readthedocs.io/en/latest/autoapi/cobra/summary/model_summary/index.html):

In [ ]:
model_rec3d.summary()

In [ ]:
for r in model_rec3d.reactions[:100]:
    gpr = r.gene_reaction_rule
    if gpr:
        print(gpr)

In [ ]:
g = model_rec3d.genes.get_by_id('55902_AT1')
g

We can display the genes from the most frequent gene (present in the largest number of reactions) to the least frequent one.

In [ ]:
d = {}

for g in model_rec3d.genes:
    d[g.name] = len(g.reactions)

In [ ]:
G = sorted(list(d.items()), key=lambda x: x[1], reverse = True)
G[:30]